**Ultralytics download**
to use TOLOv11 Model

In [ ]:
%pip install ultralytics
%pip install opencv-python
%pip install --upgrade sympy

In [2]:
import os
import random
def split_dataset(dataset_dir, train_ratio=0.8, val_ratio=0.2):

    images_dir = os.path.join(dataset_dir, "images")
    labels_dir = os.path.join(dataset_dir, "labels")
    
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg'))]
    random.shuffle(image_files)
    
    num_images = len(image_files)
    num_train = int(num_images * train_ratio)
    num_val = num_images - num_train
    
    train_images = image_files[:num_train]
    val_images = image_files[num_train:]
    
    # Create train and val directories
    train_dir = os.path.join(dataset_dir, "train")
    val_dir = os.path.join(dataset_dir, "val")
    
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(os.path.join(train_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(train_dir, "labels"), exist_ok=True)
    os.makedirs(os.path.join(val_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(val_dir, "labels"), exist_ok=True)


    # Move files to appropriate directories
    def move_files(image_list, dest_images, dest_labels):
        for image_file in image_list:
            label_file = image_file[:-4] + ".txt" # Assuming labels have .txt extensions
            
            source_image = os.path.join(images_dir, image_file)
            source_label = os.path.join(labels_dir, label_file)
            
            dest_image_path = os.path.join(dest_images, image_file)
            dest_label_path = os.path.join(dest_labels, label_file)
            
            os.rename(source_image, dest_image_path)
            if os.path.exists(source_label): # Check if the label exists.
                os.rename(source_label, dest_label_path)


    move_files(train_images, os.path.join(train_dir, "images"), os.path.join(train_dir, "labels"))
    move_files(val_images, os.path.join(val_dir, "images"), os.path.join(val_dir, "labels"))

# Example usage:
dataset_path = "./dataset" # Replace with the path to your dataset
split_dataset(dataset_path)

In [3]:
# prompt: custom dataset yaml format for yolo avoid function

import os

# Define the path to your custom dataset directory
dataset_path = "./dataset"  # Replace with the actual path

# Create the data.yaml file
data_yaml_content = f"""
path: {dataset_path}  # dataset root dir
train: train/images  # train images (relative to 'path') 128 images
val: val/images  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
names:
  1: cardboard  # Replace 'your_class_name' with the actual class name
  2: glass
  3: metal
"""

# Create the data.yaml file in the dataset directory
with open(os.path.join(dataset_path, "data.yaml"), "w") as f:
  f.write(data_yaml_content)

print(f"data.yaml file created at: {os.path.join(dataset_path, 'data.yaml')}")


data.yaml file created at: ./dataset\data.yaml


In [7]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Display model information (optional)
model.info()


100%|██████████| 5.35M/5.35M [00:04<00:00, 1.32MB/s]


YOLO11n summary: 319 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(319, 2624080, 0, 6.614336)

In [11]:
# Train the model
results = model.train(data="data.yaml", epochs=100, imgsz=640)

Ultralytics 8.3.75  Python-3.12.9 torch-2.3.1 CPU (Intel Core(TM) i7-6500U 2.50GHz)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_w

train: Scanning C:\Users\YOGA\Desktop\Master s3\NoSQL\repo\intelligent-recycling-system\datasets\dataset\train\labels... 24 images, 0 backgrounds, 0 corrupt: 100%|██████████| 24/24 [00:00<00:00, 369.27it/s]

train: New cache created: C:\Users\YOGA\Desktop\Master s3\NoSQL\repo\intelligent-recycling-system\datasets\dataset\train\labels.cache



val: Scanning C:\Users\YOGA\Desktop\Master s3\NoSQL\repo\intelligent-recycling-system\datasets\dataset\val\labels... 6 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6/6 [00:00<00:00, 1499.48it/s]

val: New cache created: C:\Users\YOGA\Desktop\Master s3\NoSQL\repo\intelligent-recycling-system\datasets\dataset\val\labels.cache


...

    100/100         0G     0.4686      1.234      0.974          8        640: 100%|██████████| 2/2 [00:29<00:00, 14.60s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.87s/it]

                   all          6          6      0.912        0.5      0.978      0.769



100 epochs completed in 0.978 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 5.5MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics 8.3.75  Python-3.12.9 torch-2.3.1 CPU (Intel Core(TM) i7-6500U 2.50GHz)
YOLO11n summary (fused): 238 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.85s/it]


                   all          6          6      0.829          1      0.995      0.853
             cardboard          5          5      0.746          1      0.995       0.81
                 glass          1          1      0.912          1      0.995      0.895
Speed: 5.4ms preprocess, 278.2ms inference, 0.0ms loss, 5.5ms postprocess per image
Results saved to runs\detect\train4


Save model

In [17]:
import torch

# Save the model
torch.save(model.state_dict(), "yolo_model.pth")

Load Model

In [19]:
# Load the saved weights
model.load_state_dict(torch.load("yolo_model.pth"))
model.eval()  # Set to evaluation mode

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): ModuleList(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
              (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
    

In [ ]:
import cv2
# Initialize the webcam
cap = cv2.VideoCapture(0)  # 0 for default webcam

while cap.isOpened():
    # Read a frame from the webcam
    success, frame = cap.read()

    if success:
        # Run inference on the frame
        results = model(frame)

        # Get the annotated frame with bounding boxes
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Object Detection", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        print("Error reading frame from webcam.")
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()




0: 480x640 2 glasss, 401.6ms
Speed: 5.6ms preprocess, 401.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 415.0ms
Speed: 24.7ms preprocess, 415.0ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 301.9ms
Speed: 3.4ms preprocess, 301.9ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 glass, 302.5ms
Speed: 3.2ms preprocess, 302.5ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 314.7ms
Speed: 3.5ms preprocess, 314.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 glass, 284.3ms
Speed: 5.5ms preprocess, 284.3ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 glass, 310.2ms
Speed: 9.1ms preprocess, 310.2ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 glasss, 321.1ms
Speed: 10.9ms preprocess, 321.1ms inference, 2.1ms post